In [ ]:
import ollama
from pydantic import BaseModel
import pandas as pd

In [ ]:
class Output(BaseModel):
  query: str
  positive: str
  negative: str

options = {
    'temperature': 0.7,
    'top-p': 0.9
}

content = """
Вывод осуществляется ТОЛЬКО на Русском языке, никаком больше

Ты продвинутый ML инженер

Твоя задача сгенерировать данные для обучения кросс энкодера. Данные должны выглядеть следующим образом:
1) генерируешь любой вопрос на абсолютно любую тему
2) генерируешь ответ на этот вопрос, правильный
3) генерируешь неправильный ответ, который должен быть на совсем другой вопрос, совершенно не связанный


Формат вывода:
Это должен быть вывод в формате JSON, который выглядит следующим образом(далее шаблон):
{{'query': Сгенерированный вопрос из пункта 1,'positive': сгенерированный правильный ответ из пункта 2, 'negative': сгенерированный неправильный ответ из пункта 3}}

НЕ ВЫВОДИ НИЧЕГО КРОМЕ ШАБЛОНА, ТОЛЬКО JSON
"""
dataset = pd.DataFrame(columns=['query','positive','negative'])
for i in range(200000):
  response = ollama.chat(model='deepseek-r1:latest',
                        messages=[{'role':'user','content':content}],
                        options=options,
                        format=Output.model_json_schema())

  output = Output.model_validate_json(response.message.content)
  dataset.loc[len(dataset)] = {'query':output.query,'positive':output.positive,'negative':output.negative}

In [ ]:
dataset.to_excel('Validation.xlsx')